+ Future development plan: 
    -   schedule direct into cobra
    

In [1]:
import pygsheets
import pandas as pd
import numpy as np
import datetime as dt
import time
import filepath
import pandas as pd
import glob
import os
import numpy as np
import pygsheets
import infrastructure
import datetime
import math
import datetime
from datetime import timedelta  
import re
import warnings
import statistics
from datetime import date
from calendar import monthrange
from dateutil.relativedelta import relativedelta
import xlsxwriter
import send_email
from colorama import Fore, Style
import matplotlib.pyplot as plt
import seaborn as sns
import filepath 
import datetime as dt
from scipy.stats import percentileofscore
from datetime import datetime
import math 

In [2]:
lanina = infrastructure.get_lanina()

In [3]:
t2=time.time()
sms = pd.read_csv('SS_LC_merged_data.csv')
df = infrastructure.transform_sms_df(sms)

# df = df.loc[:,~df.columns.duplicated()].copy()

df = df[~df['Send Strategy'].isin(['PT','RT','IT','OT','W','MI','Mi'])]
#testMaster = scheduleMaster[scheduleMaster['Send Strategy'].isin(['PT','RT','IT','OT','CT'])]
#miningMaster = scheduleMaster[scheduleMaster['Send Strategy']=='MI']
#scheduleMaster = scheduleMaster[~(scheduleMaster.index.isin(testMaster.index) | scheduleMaster.index.isin(miningMaster.index))] #create index s time series sorted index
df['Datestring']=df['Date'].dt.strftime('%m.%d.%y') #create datestring variable 
df['CTR']=(df['Clicks']/df['Delivered'])
df['eCPM']=(df['Revenue']*1000/df['Delivered'])
df['JPCTR']=(df['Jump Page Clicks']/df['Delivered'])
df['optout rate']=(df['Optout']/df['Delivered'])
df['CTR Normalized']=(df['CTR']-df['CTR'].min())/(df['CTR'].max()-df['CTR'].min())
df['eCPM Normalized']=(df['eCPM']-df['eCPM'].min())/(df['eCPM'].max()-df['eCPM'].min())
df['CTR50'] = df['CTR Normalized'] + df['eCPM Normalized']
df['Creative Type'] = df['Creative Type'].str.split(' ',expand = True)[0]
# Import Lanina
lanina = infrastructure.get_lanina()
lanina['OfferIDs'] = lanina['OfferIDs'].astype(str).str.split('.',expand = True)[0]
# mamba 
mamba = infrastructure.get_mamba()
    # oppo ctr
temp1= df.groupby(['Segments','Opportunity Cost Send Strategy','Date']).agg({'Revenue':'sum','Clicks':'sum','Delivered':'sum'}).reset_index()
temp1[['rolling Revenue','rolling Clicks','rolling Delivered']] = temp1.groupby('Segments').shift(1).rolling(30, min_periods=5)[['Revenue','Clicks','Delivered']].sum().reset_index(drop=True)
temp1['Dataset_Agg_30day_ctr'] = temp1['rolling Clicks'] / temp1['rolling Delivered']
temp1['Dataset_Agg_30day_ecpm'] = temp1['rolling Revenue'] * 1000/ temp1['rolling Delivered']
dataset_agg_ctr =  temp1[['Segments','Date','Opportunity Cost Send Strategy','Dataset_Agg_30day_ctr','Dataset_Agg_30day_ecpm']]
df = df.merge(dataset_agg_ctr, how = 'left')
df['Opportunity Clicks'] = df['Clicks'] - df['Dataset_Agg_30day_ctr'] * df['Delivered']
df['Opportunity Cost'] = df['Revenue'] - df['Dataset_Agg_30day_ecpm'] * df['Delivered'] /1000  # revenue  - last 30 days average revenue in that drop 
df['opportunity CTR'] = df['Opportunity Clicks'] / df['Delivered']
df['opportunity eCPM'] = df['Opportunity Cost'] * 1000 / df['Delivered']

# change the following code to the function calculate_oppo_ctr
# I need to calculate 
def calculate_oppo_ctr(df):
    df['opportunity CTR'] = df['Opportunity Clicks'] / df['Delivered']
    
    opportunity_ctr_series = df[(df['opportunity CTR']!=np.inf)&(df['opportunity CTR'].isna() == False)]['opportunity CTR']
    #df['opportunity CTR Normalized']=(df['opportunity CTR']-df['opportunity CTR'].min())/(df['opportunity CTR'].max()-df['opportunity CTR'].min())
    df['opportunity CTR Normalized']= percentileofscore(opportunity_ctr_series, df['opportunity CTR'], kind='weak') / 100
    df['opportunity eCPM'] = df['Opportunity Cost'] * 1000 / df['Delivered']
    opportunity_eCPM_series =  df[(df['opportunity eCPM']!=np.inf)&(df['opportunity eCPM'].isna() == False)]['opportunity eCPM']
    #df['opportunity eCPM Normalized']=(df['opportunity eCPM']-df['opportunity eCPM'].min())/(df['opportunity eCPM'].max()-df['opportunity eCPM'].min())
    df['opportunity eCPM Normalized']=percentileofscore(opportunity_eCPM_series, df['opportunity eCPM'], kind='weak') / 100 
    df['opportunity CTR50'] = df['opportunity CTR Normalized']*0.6 + df['opportunity eCPM Normalized']*0.4
    return df

df = calculate_oppo_ctr(df)


/Users/nathan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,1,2,3,4,6,8,9,10,11,12,14,16,26,27,28,29,30,31,32,33,34,35,36,37,44,48,51,54,57,58,62) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/nathan/Documents/SMS DMA Python/SMS DataSource/infrastructure.py:112: FutureWarning: The default value of regex will change from True to False in a future version.
  snakes['Hitpath Offer ID']=snakes['Hitpath Offer ID'].astype(str).str.replace('.0', '')
/Users/nathan/Documents/SMS DMA Python/SMS DataSource/infrastructure.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mamba['Affiliate ID'] = ma

# Select by Opportunity Cost CTR50 (60% Oppr CTR Normalized + 40% Oppr eCPM Normalized)

In [4]:

####################### Content Selection with HTML Selection #######################

# ignore all warnings 
import warnings
warnings.filterwarnings('ignore')


def content_select(df,sc,pubid,hitpath):
    ccid=''
    mamba = infrastructure.get_mamba()
    # according to pubid and sc to find out sc_dppub from df
    publisher = infrastructure.get_publisher()
    dppub = publisher[publisher['PUBID']==int(pubid)]['DP.DS or DP.sV'].values[0]
    dv = publisher[publisher['PUBID']==int(pubid)]['Sub Vertical'].values[0]
    sc_dppub = sc + "_" + dppub
    
    df['Opp Cost eCPM'] = df['Opportunity Cost'] / df['Delivered'] * 1000
    df = df.merge(lanina[['Reporting Content ID','Old Content ID']], how = 'left', left_on = 'Creative Type', right_on = 'Old Content ID')
    df.loc[df['Reporting Content ID'].isna() == False,'Creative Type'] = df['Reporting Content ID']
    df.drop(columns=['Reporting Content ID','Old Content ID'], inplace=True)
    # Create dataframe of all drops of given hitpath in past 90 days
    df_content = df[(df['Date'] > pd.Timestamp.today() - pd.to_timedelta(90, unit='d')) & (df['Hitpath Offer ID']==hitpath) & (df['Shortcode Name']==sc)]
    df_content = df_content[(df_content['Creative Type'].isin(lanina['Reporting Content ID'].unique().tolist())) ]

    
    df_content['shortcode_DP.SV'] = df_content['shortcode_DP.SV'] == sc_dppub 
    df_content['data_vertical'] = df_content['Data Vertical'] == dv
    df_content_by_drop = df_content.copy()
    ## get the opportunity CTR50 for each content piece in the same data vertical 
    df_content_other_affliate = df_content[(df_content['shortcode_DP.SV']==False) & (df_content['data_vertical'] == True)]
    df_content_other_affliate = df_content_other_affliate.groupby(['data_vertical','Creative Type','Hitpath Offer ID'], as_index=False).agg({'Delivered': 'sum', 'Opportunity Cost': 'sum','Opportunity Clicks': 'sum','Clicks': 'sum','Revenue':'sum'})
    df_content_other_affliate = calculate_oppo_ctr(df_content_other_affliate)
    df_content_other_affliate = df_content_other_affliate[['Creative Type','opportunity CTR50']]
    df_content_other_affliate = df_content_other_affliate.rename(columns = {'opportunity CTR50':'opportunity CTR50_data_vertical_level'})


    #offer_median_eCPM = df_content['Opp Cost eCPM'].median()
    offer_median_eCPM = df_content['opportunity CTR50'].median()
    df_content_drops = df_content.groupby(['shortcode_DP.SV','Creative Type'], as_index=False).count()
    df_content_drops = df_content_drops[['shortcode_DP.SV','Creative Type', 'Hitpath Offer ID']]
    df_content_drops.columns = ['shortcode_DP.SV','Creative Type','Drops']
    df_content = df_content.groupby(['shortcode_DP.SV','Creative Type','Hitpath Offer ID'], as_index=False).agg({'Delivered': 'sum', 'Opportunity Cost': 'sum','Opportunity Clicks': 'sum','Clicks': 'sum','Revenue':'sum'})
    df_content = calculate_oppo_ctr(df_content)
    df_content['Calculated Opp Cost eCPM'] = df_content['Opportunity Cost'] / df_content['Delivered'] * 1000
    df_content['Calculated CTR'] = df_content['Clicks'] / df_content['Delivered']
    df_content['Calculated opp CTR'] = df_content['Opportunity Clicks'] / df_content['Delivered']
    #df_content['Calculated Opp Cost eCPM'] = df_content['Opportunity Cost'] / df_content['Delivered'] * 1000
    df_content = df_content.merge(df_content_drops, on=['shortcode_DP.SV','Creative Type'])

    # Get the drops for the affiliate account
    df_content_affliate = df_content[df_content['shortcode_DP.SV']==True]
    df_content_affliate_eCPM = df_content_by_drop[df_content_by_drop['shortcode_DP.SV']==True]['opportunity CTR50'].median()
    df_content_drops = df_content_drops[df_content_drops['shortcode_DP.SV']==True]
    df_content_affliate_drops = df_content_affliate.groupby('Creative Type', as_index=False).count()
    #df_content_affliate_eCPM = df_content_affliate['Opportunity Cost'].sum() / df_content_affliate['Delivered'].sum() * 1000
    df_content_affliate = calculate_oppo_ctr(df_content_affliate)
    #df_content_affliate['Calculated Opp Cost eCPM'] = df_content_affliate['Opportunity Cost'] / df_content_affliate['Delivered'] * 1000
    df_content_affliate = df_content_affliate[['shortcode_DP.SV','Creative Type','opportunity CTR50','Calculated Opp Cost eCPM','Calculated CTR','Calculated opp CTR','Drops']]
    df_content_affliate['Meets Threshold'] =  df_content_affliate['opportunity CTR50'] >= df_content_affliate_eCPM

    # Get date last dropped in affiliate account and whether it is in the past 30 days or not
    df_content_dates = df[(df['Date'] > pd.Timestamp.today() - pd.to_timedelta(90, unit='d')) & \
                        (df['Hitpath Offer ID']==hitpath) & \
                        (df['shortcode_DP.SV'] == sc_dppub)]
    df_content_dates = df_content_dates[['Creative Type','Date']]
    df_content_dates = df_content_dates.groupby(['Creative Type'],as_index=False).max()
    cobra_dates = mamba[(mamba['Hitpath Offer ID']==hitpath) & (mamba['shortcode_DP.SV']==sc_dppub) & (mamba['Creative']!='') & (mamba['Date']> df['Date'].max() ) ][['Date','Creative']]
    cobra_dates['Creative'] = cobra_dates['Creative'].str.replace('NEW','')
    cobra_dates['Creative'] = cobra_dates['Creative'].str.replace('*','')
    cobra_dates['Creative'] = cobra_dates['Creative'].str.split('\n')
    cobra_dates = cobra_dates.explode('Creative')
    cobra_dates = cobra_dates.reset_index(drop=True)
    cobra_dates['Creative'] = cobra_dates['Creative'].str.strip()
    cobra_dates = cobra_dates[cobra_dates['Creative']!='']
    #cobra_dates = mamba[(mamba['Hitpath Offer ID']==hitpath) & (mamba['shortcode_DP.SV']==sc_dppub)][['Date','Creative']]
    cobra_dates.columns = ['Date','Creative Type']
    df_content_dates = pd.concat([df_content_dates,cobra_dates],ignore_index=True).drop_duplicates()
    df_content_dates = df_content_dates.groupby(['Creative Type'],as_index=False).max()
    df_content_dates['Recently Dropped'] = df_content_dates['Date'] > pd.Timestamp.today() - pd.to_timedelta(30, unit='d')

    # Get the drops for all other accounts
    df_content_other = df_content[df_content['shortcode_DP.SV']==False]
    df_content_other_eCPM = df_content_by_drop[df_content_by_drop['shortcode_DP.SV']==False]['opportunity CTR50'].median()
    df_content_drops = df_content_drops[df_content_drops['shortcode_DP.SV']==False]
    #df_content_other_eCPM = df_content_other['Opportunity Cost'].sum() / df_content_other['Delivered'].sum() * 1000
    df_content_other = calculate_oppo_ctr(df_content_other)
    df_content_other = df_content_other.merge(df_content_other_affliate, on = 'Creative Type', how = 'left') 
    df_content_other.loc[df_content_other['opportunity CTR50_data_vertical_level'].isna() == False, 'opportunity CTR50'] = df_content_other['opportunity CTR50_data_vertical_level'] * 0.7 +   df_content_other['opportunity CTR50'] * 0.3
    
    #df_content_other_median_eCPM = df_content_other['Revenue CPM (eCPM)'].median()
    df_content_other = df_content_other[['shortcode_DP.SV','Creative Type','opportunity CTR50','Calculated Opp Cost eCPM','Calculated CTR','Calculated opp CTR','Drops']]
    df_content_other['Meets Threshold'] =  df_content_other['opportunity CTR50'] >= df_content_other_eCPM
  
 


    # Merge the above dataframes together 
    df_content = df_content_other.merge(df_content_affliate, how='outer', on='Creative Type').merge(df_content_dates, how='outer', on='Creative Type')
    df_content['Meets Threshold'] = (df_content['Meets Threshold_y'] == True) | ((df_content['Meets Threshold_y'].isnull()) & (df_content['Meets Threshold_x'] == True))
    print('Offer eCPM for Other Accounts: ', round(df_content_other_eCPM,2))
    print('Offer eCPM for Affiliate Account: ', round(df_content_affliate_eCPM,2))
    print('Offer median eCPM: ',round(offer_median_eCPM,2))

    df_content = df_content[['Creative Type', 'opportunity CTR50_x', 'Drops_x',
                                'opportunity CTR50_y', 'Drops_y', 'Date', 'Recently Dropped', 'Meets Threshold']]
    df_content.columns = ['Creative Type','eCPM in Other Accounts','Drops in Other Accounts',
                            'eCPM in Affiliate Account', 'Drops in Affiliate Accounts','Date','Recently Dropped',
                            'Meets Mean Threshold']
    # check if the content has a median eCPM higher than the offer median eCPM
    df_content_by_drop = df_content_by_drop.groupby(['Creative Type'], as_index=False)['opportunity CTR50'].median()
    df_content = df_content.merge(df_content_by_drop, how='left', on='Creative Type')
    df_content = df_content.rename(columns = {'opportunity CTR50':'median eCPM'})
    df_content['Meets Median Threshold'] = df_content['median eCPM'] >= offer_median_eCPM

    # check if the content has an aggregate eCPM in the affiliate account higher than the offer aggregate eCPM in the affiliate account 
    df_content.loc[df_content['eCPM in Affiliate Account'] >= df_content_affliate_eCPM,'Meets Median Threshold'] = True

    # rank is calculated as the average of the aggregate eCPM of the content piece and two times the median eCPM of the content piece 
    df_content['rank'] = (df_content['eCPM in Other Accounts'] + 2 * df_content['median eCPM']) / 3
    # if the content piece has been dropped in the affiliate account in the past 90 days, that aggregate eCPM will be used in the ranking instead of the aggregate for all accounts
    df_content.loc[~df_content['eCPM in Affiliate Account'].isna(),['rank']] = (df_content['eCPM in Affiliate Account'] + 2*df_content['median eCPM'])/3
    df_content = df_content.sort_values('rank', ascending=False)
    df_content = df_content.merge(lanina[['Reporting Content ID','Content Approval Status','Content']], how='left', left_on='Creative Type',right_on = 'Reporting Content ID', copy = False )
    # drop reporting content id column
    df_content = df_content.drop(columns=['Reporting Content ID'])
    # filter out content that is not approved and failed testing 
    df_content = df_content[~(df_content['Content Approval Status'].str.contains('Paused|Not Approved|Failed Testing', na = False)) ]
    df_content_all = df_content.copy()
    if df_content.empty:
        
                    # find Reporting Content ID from la nina based on offer IDs and Type 
        lanina1 =  lanina[~(lanina['Content Approval Status'].str.contains('Paused|Not Approved|Failed Testing', na = False)) ]
        df_content= lanina1[(lanina1['OfferIDs'] == hitpath) & (lanina1['Type'] == sc) & (lanina1['Channel'] =='SC')][['Reporting Content ID','Content Approval Status', 'Content']]
        if len(df_content) > 0: 
            print('No content available from content selection script, we choose 1 random content from Lanina')
            ccid  = df_content['Reporting Content ID'].values[0]
            df_content_all = df_content.copy()
            print(ccid)
        else:
            
            print('No content available from lanina')
    else:
        # select only content that has been dropped more than 2 times in any account, or has been dropped at least once in the affiliate account it
        df_content['Meets Drop Threshold'] = (df_content['Drops in Other Accounts']>=2) 
        df_content = df_content[((df_content['Meets Drop Threshold']==True) & (df_content['eCPM in Affiliate Account'].isna())) | (~df_content['eCPM in Affiliate Account'].isna())]
        df_content = df_content[(df_content['Meets Mean Threshold']==True) & (df_content['Meets Median Threshold']==True)].sort_values('rank', ascending=False)

    # if no content meets threshold, remove drop number requirement
        if df_content.empty:
            df_content = df_content_all.copy()
            df_content = df_content[(df_content['Meets Mean Threshold']==True) & (df_content['Meets Median Threshold']==True)].sort_values('rank', ascending=False)

        # if after remove number of drop restriction no content meets threshold, reduce mean and median threshold until content is found
        k=0
        while df_content.empty:
            k = k + 1
            print('No offers, decreasing thresholds.')
            df_content = df_content_all.copy()
            df_content_other['Meets Threshold'] =  df_content_other['opportunity CTR50'] >= df_content_other_eCPM - k * 0.1
            df_content_affliate['Meets Threshold'] =  df_content_affliate['opportunity CTR50'] >= df_content_affliate_eCPM - k * 0.1
            df_content['Meets Median Threshold'] = df_content['median eCPM'] >= offer_median_eCPM - k * 0.1
            df_content = df_content[(df_content['Meets Mean Threshold']==True) & (df_content['Meets Median Threshold']==True)].sort_values('rank', ascending=False)

            if k > 20:
                df_content = df_content_all.copy()
                df_content['Meets Drop Threshold'] = (df_content['Drops in Other Accounts'] > 2) 
                df_content = df_content.sort_values('rank', ascending=False)
                break

        df_content = df_content.reset_index(drop=True)
        # If not specifying HTML, remove dates from HTML so gapping is not considered
        
        # Select content, either a random content piece that has not dropped in the past 30 days weighted by the rank or 4 times the rank if opp cost is positive
        # or the one least recently dropped if all have been dropped in the past 30 days
        content = df_content[df_content['Recently Dropped']!=True] # didn't drop in the past 30 days 
        #content.loc[content['rank']>0,'rank'] = content['rank'] * 4 
            
        if content.empty:
            content = df_content[(df_content['Meets Mean Threshold']==True) & (df_content['Meets Median Threshold']==True)]
            selected_creative = content[content['Date'] == content['Date'].min()][['Creative Type']]
        else:
            selected_creative = content[['Creative Type']].sample(n=1,weights=pd.to_numeric(content['rank']) + abs(2*min(pd.to_numeric(content['rank']))))
            #selected_creative = content[['Creative Type']]
        try: 
            ccid = selected_creative['Creative Type'].iloc[0]
        except: 
            ccid = df_content_all['Creative Type'].iloc[0]
        

        
        print('')
        
        print(ccid)
        print('Ranked', df_content.index[df_content['Creative Type'] == ccid][0] + 1, 'out of', len(df_content), 'pieces that meet thresholds')

    return df_content, df_content_all, ccid 



In [5]:
""" 
passed_content = lanina[lanina['Content Approval Status'].str.contains("Approved - Passed")]
df1 = df[(df['Creative Type'].isin(passed_content['Reporting Content ID'].unique().tolist())) | (df['Hitpath Offer ID']==6272)]
df2 = df1.groupby(['Creative Type'])['Date'].count().reset_index(name = "count")
temp = df2[df2['count'] == 1].merge(lanina[['Reporting Content ID','OfferIDs','Content Approval Status','Offer Status']], left_on = 'Creative Type',right_on = 'Reporting Content ID', how = 'left', copy = False).drop_duplicates().sort_values('OfferIDs', ascending = False).drop(columns = ['Reporting Content ID']).reset_index(drop = True)
temp = temp[temp['Offer Status'] == 'Passed Test - In Production']
temp.to_csv('/Users/liliguo/Desktop/passed_content.csv', index = False) 
"""

' \npassed_content = lanina[lanina[\'Content Approval Status\'].str.contains("Approved - Passed")]\ndf1 = df[(df[\'Creative Type\'].isin(passed_content[\'Reporting Content ID\'].unique().tolist())) | (df[\'Hitpath Offer ID\']==6272)]\ndf2 = df1.groupby([\'Creative Type\'])[\'Date\'].count().reset_index(name = "count")\ntemp = df2[df2[\'count\'] == 1].merge(lanina[[\'Reporting Content ID\',\'OfferIDs\',\'Content Approval Status\',\'Offer Status\']], left_on = \'Creative Type\',right_on = \'Reporting Content ID\', how = \'left\', copy = False).drop_duplicates().sort_values(\'OfferIDs\', ascending = False).drop(columns = [\'Reporting Content ID\']).reset_index(drop = True)\ntemp = temp[temp[\'Offer Status\'] == \'Passed Test - In Production\']\ntemp.to_csv(\'/Users/liliguo/Desktop/passed_content.csv\', index = False) \n'

In [6]:
        
def content_schedule(sc_dppub_affiliate, days_ahead, update_bool = False): 
    lyst = [] 
    for i in range(1,math.ceil(days_ahead/7)+1):
        lyst.append(i*7)
    for days_ahead in lyst:  
        begin_day = days_ahead-7
        gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
        cobra_google_sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/12vqSDueybprNphtsw7gXR5vmgcPG6_5ZNcnWzNpiasY/edit#gid=1186670009')
        cobra_sheet =  cobra_google_sheet.worksheet('title','New Mamba')
        cobra = infrastructure.get_mamba_full_slot()
        cobra_sheet_df = cobra_sheet.get_as_df()
        cobra_sheet_df.columns=cobra_sheet_df.iloc[0]
        
        sc = sc_dppub_affiliate.split('_')[0]
        dppub = sc_dppub_affiliate.split('_')[1]
        pubid = sc_dppub_affiliate.split('_')[2]
        sc_dppub = sc + "_" + dppub
          
        content_df=cobra[(cobra['Dataset']==sc_dppub_affiliate)& (cobra['Date'].dt.date>=date.today()+timedelta(days=begin_day)) &
                                    (cobra['Date'].dt.date<=date.today()+timedelta(days=days_ahead))]
                # set row 1 as column 
                #

        days=pd.to_datetime(content_df['Date'].unique())

        drops=['Drop 1','Drop 2','Drop 3','Drop 4','Drop 5','Drop 6']
        cols=['Time','Segment ','Send Strategy','Offer','Limit',
                    'Offset','Creative','Job Name','Blank']

        write_dict={}
        
        for index,day in enumerate(pd.to_datetime(content_df['Date'].unique()).strftime('%D')):
            for drop in drops:
                for col in cols:
                    if len(content_df[(content_df["Drop"]==drop) & (content_df['Date']==day)])>0:
                        if col == 'Blank':
                            write_dict[(drop,col)] = ''
                        else: 
                            write_dict[(drop,col)] = content_df[(content_df["Drop"]==drop) & (content_df['Date']==day)][col].values[0]
                        if col == 'Offer' and write_dict[(drop,col)] != '':
                            hitpath = write_dict[(drop,col)].split(' ')[0]
                        if col == 'Creative' and write_dict[(drop,col)] == '' and write_dict[(drop,'Offer' )] != '':
                            write_dict[(drop,col)] = content_select(df,sc,pubid,hitpath)[2]
                        if col == 'Job Name' and write_dict[(drop,col)] == '' and write_dict[(drop,'Offer' )] != '':
                                date1 = datetime.strptime(day, "%m/%d/%y").strftime("%d%b%y")  
                                #Format: SS_HZB_TLG-PL-30DC-VZN_12386_P_21Oct23    
                                write_dict[(drop,col)] =   "SS_"+write_dict[(drop,'Segment ')][:3] + "_"+write_dict[(drop,'Segment ')][4:].replace(".",'-').replace("_",'-')+"_"+ write_dict[(drop,'Offer')].split(' ')[0] +"_"+ write_dict[(drop,'Send Strategy')] + "_" +   date1       

                    else:
                        write_dict[(drop,col)] = ''

            if index==0:
                write_df = pd.DataFrame(write_dict,index=[day]).T
                write_df1 = pd.DataFrame(write_dict,index=[day]).T

            else:
                write_df_new = pd.DataFrame(write_dict,index=[day]).T
                write_df = pd.concat([write_df,write_df_new],axis=1)

            #write_df=write_df.replace(np.nan,'')
        if update_bool == True:
                    
            cobra_row = int(cobra_sheet_df[cobra_sheet_df['Dataset'].str.contains(sc_dppub_affiliate)].index.values[0]+3)
            if len(content_df) >0:
                date_string = content_df['Date'].dt.date.values[0].strftime('%-m/%-d/%Y')
                #3.11.5 python version accept this: date_string = content_df['Date'].dt.date.values[0].strftime('%#m/%#d/%Y')
                cobra_column = cobra_sheet_df.columns.get_loc(date_string)+1
                cobra_sheet.set_dataframe(start=(cobra_row,cobra_column),df=write_df,copy_head=False,nan='')
                print(write_df)
        else: 
            return write_df
        


In [7]:
# sc_dppub_affiliates = ['HZB_CM.OSR_461452','MBC_CM.OSR_461452','SVT_AMD.PL_461810','HZB_AMD.PL_461810','SVT_AL.PL.3_461838','HZB_AL.PL.3_461838','DSS_TLG.PL_461768','HZB_TLG.PL_461768','MBC_WWM.YFA.2_461680','FLC_WWM.YFA.2_461680'] #the sc_dppub_affiliate for which to schedule (list of strings)
sc_dppub_affiliates =  [
    "FLC_I.CC_460918", "MBC_I.CC_460918", "FLC_SPK.CR_460921",
    "MBC_SPK.CR_460921", "FLC_AI.CC_460939", "MBC_AI.CC_460939", "HZB_RHD.CC_461263",
    "DSS_RHD.CC_461263","UAA_SPK.CR2_461842","UAATF_SPK.CR2_461842","MBC_PN.FC_461653","FLC_PN.FC_461653",
    "SVT_AL.PL_461794","HZB_AL.PL_461794","SVT_AL.PL.2_461795","HZB_AL.PL.2_461795",
    "DSS_JET.ZTA_461835","FLC_JET.ZTA_461835", "FLC_SPK.CR2_461842","MBC_PN.SWP_461500", "FLC_PN.SWP_461500", 'FLC_EDM.247L_461227','MBC_EDM.247L_461227']
# sc_dppub_affiliates = ['SVT_AL.PL.2_461795']
days_ahead = 20
update_bool = True 
for sc_dppub_affiliate in sc_dppub_affiliates: 
    try:
        content_schedule(sc_dppub_affiliate, days_ahead, update_bool)
    except Exception as e:
        print(f"An error occurred with {sc_dppub_affiliate}: {e}")


Offer eCPM for Other Accounts:  0.36
Offer eCPM for Affiliate Account:  0.63
Offer median eCPM:  0.39

13221.SC.FLC.454071
Ranked 1 out of 1 pieces that meet thresholds
Offer eCPM for Other Accounts:  0.61
Offer eCPM for Affiliate Account:  0.53
Offer median eCPM:  0.6

12710.SC.FLC.453521
Ranked 1 out of 1 pieces that meet thresholds
Offer eCPM for Other Accounts:  0.53
Offer eCPM for Affiliate Account:  0.75
Offer median eCPM:  0.53

VAST.6322.SC.FLC-CT.1
Ranked 1 out of 1 pieces that meet thresholds
Offer eCPM for Other Accounts:  0.74
Offer eCPM for Affiliate Account:  0.55
Offer median eCPM:  0.73

UG.11600.SC.FLC.451332
Ranked 1 out of 2 pieces that meet thresholds
                                             05/23/24  \
Drop 1 Time                              10:00 AM PST   
       Segment                          FLC_I.CC_30DC   
       Send Strategy                                P   
       Offer                        8838 - W4 EDU FLC   
       Limit                       

Offer eCPM for Other Accounts:  0.3
Offer eCPM for Affiliate Account:  0.18
Offer median eCPM:  0.29

13113.SC.FLC.453641
Ranked 1 out of 1 pieces that meet thresholds
Offer eCPM for Other Accounts:  0.42
Offer eCPM for Affiliate Account:  0.12
Offer median eCPM:  0.42

CGC.11646.SC.FLC.451972
Ranked 1 out of 3 pieces that meet thresholds
Offer eCPM for Other Accounts:  0.3
Offer eCPM for Affiliate Account:  nan
Offer median eCPM:  0.3

RU.11793.SC.FLC-P.1
Ranked 1 out of 1 pieces that meet thresholds
                                              05/30/24  \
Drop 1 Time                               10:00 AM PST   
       Segment                           FLC_I.CC_30DC   
       Send Strategy                                 P   
       Offer                         11600 - WI UG FLC   
       Limit                                             
       Offset                                            
       Creative                 UG.11600.SC.FLC.451332   
       Job Name       SS_FLC_

                     06/06/24 06/07/24 06/08/24 06/09/24 06/10/24 06/11/24  \
Drop 1 Time                                                                  
       Segment                                                               
       Send Strategy                                                         
       Offer                                                                 
       Limit                                                                 
       Offset                                                                
       Creative                                                              
       Job Name                                                              
       Blank                                                                 
Drop 2 Time                                                                  
       Segment                                                               
       Send Strategy                                            

Offer eCPM for Other Accounts:  0.66
Offer eCPM for Affiliate Account:  nan
Offer median eCPM:  0.66

RU.11793.SC.MBC.451332
Ranked 1 out of 1 pieces that meet thresholds
Offer eCPM for Other Accounts:  0.46
Offer eCPM for Affiliate Account:  0.35
Offer median eCPM:  0.45

12710.SC.MBC.453861
Ranked 1 out of 3 pieces that meet thresholds
Offer eCPM for Other Accounts:  0.58
Offer eCPM for Affiliate Account:  0.21
Offer median eCPM:  0.57

CGC.11646.SC.MBC.451972
Ranked 1 out of 1 pieces that meet thresholds
                                                 05/30/24  \
Drop 1 Time                                  10:00 AM PST   
       Segment                          MBC_I.CC_30DC_TMO   
       Send Strategy                                    P   
       Offer                           6322 - BB VAST MBC   
       Limit                                                
       Offset                                               
       Creative                      VAST.6322.SC.MBC-P.1   

                     06/06/24 06/07/24 06/08/24 06/09/24 06/10/24 06/11/24  \
Drop 1 Time                                                                  
       Segment                                                               
       Send Strategy                                                         
       Offer                                                                 
       Limit                                                                 
       Offset                                                                
       Creative                                                              
       Job Name                                                              
       Blank                                                                 
Drop 2 Time                                                                  
       Segment                                                               
       Send Strategy                                            

...

                     05/30/24 05/31/24 06/01/24 06/02/24 06/03/24 06/04/24  \
Drop 1 Time                                                                  
       Segment                                                               
       Send Strategy                                                         
       Offer                                                                 
       Limit                                                                 
       Offset                                                                
       Creative                                                              
       Job Name                                                              
       Blank                                                                 
Drop 2 Time                                                                  
       Segment                                                               
       Send Strategy                                            

Offer eCPM for Other Accounts:  0.72
Offer eCPM for Affiliate Account:  0.77
Offer median eCPM:  0.75

MLUSA.12588.SC.FLC.452231
Ranked 1 out of 1 pieces that meet thresholds
Offer eCPM for Other Accounts:  0.44
Offer eCPM for Affiliate Account:  nan
Offer median eCPM:  0.44

13074.SC.FLC.453901
Ranked 1 out of 1 pieces that meet thresholds
Offer eCPM for Other Accounts:  0.5
Offer eCPM for Affiliate Account:  0.41
Offer median eCPM:  0.45

13194.SC.FLC.453941
Ranked 1 out of 1 pieces that meet thresholds
Offer eCPM for Other Accounts:  0.43
Offer eCPM for Affiliate Account:  0.33
Offer median eCPM:  0.43

12972.SC.FLC.453841
Ranked 2 out of 3 pieces that meet thresholds
                                                        05/23/24  \
Drop 1 Time                                         10:00 AM PST   
       Segment                            FLC_EDM.247L_30DC_NEXL   
       Send Strategy                                           P   
       Offer                                   1

Offer eCPM for Other Accounts:  0.23
Offer eCPM for Affiliate Account:  nan
Offer median eCPM:  0.23

13253.SC.FLC.454251
Ranked 1 out of 1 pieces that meet thresholds
Offer eCPM for Other Accounts:  0.32
Offer eCPM for Affiliate Account:  0.12
Offer median eCPM:  0.29

13113.SC.FLC.453641
Ranked 1 out of 1 pieces that meet thresholds
Offer eCPM for Other Accounts:  0.76
Offer eCPM for Affiliate Account:  0.43
Offer median eCPM:  0.73

UG.11600.SC.FLC.451631
Ranked 1 out of 3 pieces that meet thresholds
                                                       05/30/24  \
Drop 1 Time                                        10:00 AM PST   
       Segment                           FLC_EDM.247L_30DC_NEXL   
       Send Strategy                                          P   
       Offer                                        12972 - FLC   
       Limit                                                      
       Offset                                                     
       Creative       

                     06/06/24 06/07/24 06/08/24 06/09/24 06/10/24 06/11/24  \
Drop 1 Time                                                                  
       Segment                                                               
       Send Strategy                                                         
       Offer                                                                 
       Limit                                                                 
       Offset                                                                
       Creative                                                              
       Job Name                                                              
       Blank                                                                 
Drop 2 Time                                                                  
       Segment                                                               
       Send Strategy                                            

                     05/30/24 05/31/24 06/01/24 06/02/24 06/03/24 06/04/24  \
Drop 1 Time                                                                  
       Segment                                                               
       Send Strategy                                                         
       Offer                                                                 
       Limit                                                                 
       Offset                                                                
       Creative                                                              
       Job Name                                                              
       Blank                                                                 
Drop 2 Time                                                                  
       Segment                                                               
       Send Strategy                                            